# Initial setup

## Load the LiCS-Auto-Batch module

First step is to load the auto batch module:
```batch
module load licsar_framebatch
```

## Cache setup
You will need to create a directory which you intend to use as a processing cache (for all frames). This should be in the group workspace . You then need to export the path to this directory as an environment variable:
```bash
export BATCH_CACHE_DIR=.../path/to/cache
```
The cache is a *temporary* workspace and is intended to store **all** intermediate products - this ensures that all jobs can be repeated if required.

## Setup temporary dir
If you don't have the following directory, create:
```bash
mkdir /gws/nopw/j04/nceo_geohazards_vol1/insar_temp/$USER
```

## Log directory setup
I would strongly suggest you create a directory structure for individual frames where you dispatch LOTUS jobs from. Lotus output and error files will then be written here - this makes it easier to find things when they go wrong. This will be quite small, so you can put this in your home directory.

## Get access to the processing spreadsheet
The batch database is mirrored to the [processing spreadsheet](https://docs.google.com/spreadsheets/d/1Rbt_nd5nok-UZ7dfBXFHsZ66IqozrHwxiRj-TDnVDMY/edit?usp=sharing). You will require access to this spreadsheet to do the processing.

## Activating/Deactivating a Frame

By default your frame will be inactive. To see in the spreadsheet, you must activate it first:
```
setFrameActive.py <frame>
```
Conversely you can set a frame inactive (do this) when you no longer need to run jobs on a frame:
```
setFrameInactive.py <frame>
```

# Processing

## Initialise a frame for processing
The first step in processing is to setup a individual frame with a cache, jobs and products. This can be done in a single command:
```batch
createFrameCache.py <FRAME> <BATCH_N>
```
<FRAME> is the frame name you wish to setup and <BATCH_N> is the number of jobs you wish to use to create a set of products - i.e. if BATCH_N=20 you will create 20 jobs for all of you slc's,20 jobs for your rslcs, etc...

I'm currently using a batch size of 20.

After setup, you should find your frame in the Frames sheet of the processing spreadsheet.

## Request files from the nla
Request files from the nla as you would normally do, using the `LiCSAR_0_getFiles.py` tool.

Example: 
```
mkdir $BATCH_CACHE_DIR/$FRAME
LiCSAR_0_getFiles.py -f $FRAME -s 2014-10-01 -e $(date +%Y-%m-%d) -r -b Y -n -z $BATCH_CACHE_DIR/$FRAME/db_query.list
nla.py requests
```
(the db_query.list file can be later used to generate connections figure)

## 

## Jobs
The batch processing is ordered into a collection of jobs. A single job represents a single batch command - The command depends on the job type. 

| job type | produces.. |
| ---- | ---- |
| mk_imag | slc |
| coreg | rslc |
| mk_ifg | ifg |
| unwrap |  unwrapped ifg |

You can find all the jobs in the job sheet. To filter the job sheet create a new [filter view](https://support.google.com/docs/answer/3540681?hl=en&co=GENIE.Platform=Desktop), **DO NOT FILTER THE SHEET DIRECTLY**.

Jobs are linked to products. To find products, look at the corresponding sheet for the products. (e.g. slc for slc's). Notice that each individual product is linked to a job, and shows it's current status and job status (as well as any required product status).

When a job is ran, it will process all products which are linked to it which are i) not built and ii) have the required previous products. e.g. if you run a mk_ifg job, it will only try to make interferograms which havn't been made and have both rslc's available. This means that you can spam jobs safely! 

Each product is also made in its own separate temporary directory, under `/gws/nopw/j04/nceo_geohazards_vol1/insar_temp/$USER/<frame>_envs/`. This ensures that your cache directory does not get corrupted!

### Running Jobs

To run a job, first make sure you are in your frames working directory and that you have the correct module and environment variables set. Then go to the Job Commands sheet. Select the job commands you wish to run, copy and paste into a terminal. **Remember** you can use a filter view to create a list of commands to copy - You should be able to copy and run multiple commands at once!

#### Few tips and tricks

- run the job commands from some frame log directory to prevent flooding by generated log files


### Monitoring Jobs

The processing spreadsheet will be updated every 15 minutes. Jobs should have a status of running and individual products will have a live status (i.e. you can see which products are being built, failure codes and built products) during processing.

You can also see the status of lotus jobs concisely using:
```bash
bjobs -oa "jobid user stat job_name"
```

To kill all jobs of a particular type:
```bash
bkill -J "mk_image_*"
bkill -J "coreg_*"
bkill -J "mk_ifg_*"
bkill -J "unwrap_*"
```

## Baseline calculation

You can calculate the baseline for a frame:
```
ab_LiCSAR_baseline.py <FRAME>
```

## Clean slc's from a frame

This should happen automagically but:
```
ab_LiCSAR_clean.py
```